In [1]:
import numpy as np
import time
from sklearn.datasets import fetch_20newsgroups 

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer 

from sklearn.linear_model import SGDClassifier 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [2]:
categories = ['alt.atheism', 'soc.religion.christian', 
              'comp.graphics', 'sci.med']

# currently looking at all categories 
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, remove=('headers', 'footers', 'quotes'))
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, remove=('headers', 'footers', 'quotes'))


In [3]:
ensembleModels_names = ['AdaBoost']#we can add bagging here if we want later

ensembleModels = [
    AdaBoostClassifier(random_state=0)
]

baseEstimators_names = ['SVM', 'Logistic Regression', 'Random Forest','Decision Tree']

#add the optimal paramaters once we finish our gridSearch
baseEstimators = [
    LinearSVC(C=80,penalty = 'l2',loss = 'squared_hinge',random_state=0),
    LogisticRegression(C = 60, penalty = 'l2', solver = 'saga', random_state=0),
    RandomForestClassifier(n_estimators = 700, min_samples_split = 10, max_depth=None, max_features = 'auto', criterion = 'gini',bootstrap = False, random_state=0),
    DecisionTreeClassifier(max_depth= None, max_features = None, min_impurity_decrease= 0.0005,random_state=0)
]

In [4]:
def runAdaBoost(parameter_grids, models, model_names):
    start = time.time()
    i = 0
    best_scores = []
    best_params = []
    for model in models: 
        print("Currently training model: ", model_names[i])

        text_clf = Pipeline([('vect', CountVectorizer()),
                            ('tfidf', TfidfTransformer()),
                            ('clf', model)])

        _  = text_clf.fit(twenty_train.data, twenty_train.target)    

        gs_clf = GridSearchCV(text_clf, parameter_grids[i], n_jobs=-1, cv=3,error_score=0.0)
        gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

        best_scores.append(gs_clf.best_score_)
        best_params.append(gs_clf.best_params_)

        print("Time taken: ", time.time()-start)
        print("Best score : ", gs_clf.best_score_)
        print("Best params: ", gs_clf.best_params_)
        print("\n\n")

        i+=1
    

### Linear AdaBoost 

In [7]:
#adjust the vect/tfidf to be optimal for each model (stop words, ngram)
i = 0
Ada_linear_svc = [
    {
        'vect__ngram_range': [(1,2)],
        'vect__stop_words': ['english'],
        'tfidf__use_idf': [True],
        'clf__base_estimator': [baseEstimators[i]], #adjust the index to switch to other base_estimators
        
        #AdaBoost Paramaters
        'clf__algorithm': ['SAMME'], #SVC is a strong learner, so this is needed, possibly will overfit
        'clf__n_estimators': [50, 100],
        'clf__learning_rate' : [0.01,0.05,0.1,0.3,1],
    }
]
print(baseEstimators[i])

LinearSVC(C=80, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)


In [ ]:
runAdaBoost(Ada_linear_svc, [ensembleModels[0]], [ensembleModels_names[0]])

Currently training model:  AdaBoost


### Logistic Regression AdaBoost

In [5]:
#adjust the vect/tfidf to be optimal for each model (stop words, ngram)
i=1
Ada_Logistic_Reg = [
    {
        'vect__ngram_range': [(1,2)],
        'vect__stop_words': ['english'],
        'tfidf__use_idf': [True],
        'clf__base_estimator': [baseEstimators[i]], #adjust the index to switch to other base_estimators
        
        #AdaBoost Paramaters
        'clf__n_estimators': [50],
        'clf__learning_rate' : [1],
    }
]
print(baseEstimators[i])

LogisticRegression(C=60, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)


In [6]:
runAdaBoost(Ada_Logistic_Reg, [ensembleModels[0]], [ensembleModels_names[0]])

Currently training model:  AdaBoost


C:\Users\heath\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\heath\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


KeyboardInterrupt: 

### Random Forest

In [9]:
i = 2
Ada_Random_Forest = [
    {
        'vect__ngram_range': [(1,2)],
        'vect__stop_words': ['english'],
        'tfidf__use_idf': [True],
        'clf__base_estimator': [baseEstimators[i]], #adjust the index to switch to other base_estimators
        
        #AdaBoost Paramaters
        'clf__n_estimators': [50],
        'clf__learning_rate' : [1],
    }
]
print(baseEstimators[i])

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=700,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)


In [ ]:
runAdaBoost(Ada_Random_Forest, [ensembleModels[0]], [ensembleModels_names[0]])

### Decision Tree

In [8]:
i = 3
Ada_Decision_Tree = [
    {
        'vect__ngram_range': [(1,2)],
        'vect__stop_words': [None],
        'tfidf__use_idf': [False],
        'clf__base_estimator': [baseEstimators[i]], #adjust the index to switch to other base_estimators
        
        #AdaBoost Paramaters
        'clf__n_estimators': [50],
        'clf__learning_rate' : [1],
    }
]
print(baseEstimators[i])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0005, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')


In [ ]:
runAdaBoost(Ada_Decision_Tree, [ensembleModels[0]], [ensembleModels_names[0]])